In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import csv
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForMaskedLM

#Encoder-Decoder Model
from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer
from transformers import Trainer
from evaluate import load
#Training

from transformers import TrainingArguments
from transformers import  Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

from datasets import Dataset
import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss, MSELoss
# bertscore = load("bertscore") 
tokenizer =  AutoTokenizer.from_pretrained("t5-small",cahe_dir='/ssd_scratch/cvit/aparna/t5_small')
model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/t5_simplification_final")

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import pandas as pd
from tqdm import tqdm
colnames = ['source', 'target']
input_file = "../../data/10/train.tsv"
test_data = pd.read_csv(input_file, sep="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', header=None, names=colnames)
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
#test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1


for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   
    for i, beam in enumerate(beam_outputs):
        x = pd.DataFrame([[text,tokenizer.decode(beam, skip_special_tokens=True)]],columns=["source","target"])
        test_data = pd.concat([test_data,x],ignore_index=True)
        #print(test_data.shape)
        if test_data.shape[0]%1000 ==0:
            test_data.to_csv("t5_simplification_final_10k.csv",index=False)
            print("Saved")
      


  0%|          | 0/483801 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  0%|          | 20/483801 [00:32<313:45:23,  2.33s/it]

Saved


  0%|          | 120/483801 [03:02<381:40:47,  2.84s/it]

Saved


  0%|          | 220/483801 [05:29<303:06:16,  2.26s/it]

...

Saved


  1%|▏         | 6720/483801 [2:47:59<699:49:17,  5.28s/it]

Saved


  1%|▏         | 6813/483801 [2:51:54<200:34:58,  1.51s/it]


KeyboardInterrupt: 